This notebook visualizes the exploratory data analysis. 
Specifically the UMAP and PCA plots of the data.

In [1]:
suppressWarnings(suppressPackageStartupMessages(library(ggplot2)))
suppressWarnings(suppressPackageStartupMessages(library(cowplot)))
suppressWarnings(suppressPackageStartupMessages(library(dplyr)))
suppressWarnings(suppressPackageStartupMessages(library(arrow)))
suppressWarnings(suppressPackageStartupMessages(library(argparse)))
suppressWarnings(suppressPackageStartupMessages(library(gifski)))
if (!requireNamespace("gganimate", quietly = TRUE)) {
  install.packages("gganimate")
}
suppressWarnings(suppressPackageStartupMessages(library(gganimate)))

In [2]:
# argparser <- ArgumentParser()
# argparser$add_argument("--dataset", help="Input file", required=TRUE)

# args <- argparser$parse_args()

# data_set <- args$dataset

data_set <- "first_time"

In [3]:
input_data_path <- file.path("..","data",data_set)

In [4]:
# set the path to the data to visualize
umap_data_path <- file.path(input_data_path,"umap_embeddings.parquet")
pca_data_path <- file.path(input_data_path,"pca_embeddings.parquet")
scree_data_path <- file.path(input_data_path,"scree_plot.parquet")

# read the data
umap_data <- arrow::read_parquet(umap_data_path)
pca_data <- arrow::read_parquet(pca_data_path)
scree_data <- arrow::read_parquet(scree_data_path)

In [5]:
# sample from well, fov, and time 
dim(umap_data)
umap_data <- umap_data %>% 
  group_by(Metadata_Well, Metadata_FOV, Metadata_Time) %>% 
  sample_n(4) %>% 
  ungroup()
dim(umap_data)

[1] 3820888      35

[1] 32892    35

In [6]:
head(umap_data)

Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_BF,Metadata_Image_PathName_CL488,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1426.31944,1051.7531,56443,4.50723457,0.8430272
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1278.15894,451.1779,56443,-1.14765608,-0.8359004
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,116.34923,683.6766,56443,0.05474161,-0.2473071
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,399.61502,1031.3970,56443,-0.25203672,0.2765246
Media,C04,533,0001,Media,01,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1

In [7]:
# map the timepoints to the actual hour timepoint
timepoints <- data.frame(
    reference = c("00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17"),
    Metadata_timepoint  = c(1,4,7,10,13,16,19,22,25,28,31,34,37,40,43,46,49,90)
)
# map the timepoints to the main df 
umap_data <- umap_data %>% left_join(timepoints, by = c("Metadata_Time" = "reference"))
umap_data$Metadata_timepoint <- as.numeric(umap_data$Metadata_timepoint)
umap_data$Metadata_timepoint_hours <- paste0(umap_data$Metadata_timepoint, " hours")
head(umap_data)

Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1,Metadata_timepoint,Metadata_timepoint_hours
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1426.31944,1051.7531,56443,4.50723457,0.8430272,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1278.15894,451.1779,56443,-1.14765608,-0.8359004,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,116.34923,683.6766,56443,0.05474161,-0.2473071,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,399.61502,1031.3970,56443,-0.25203672,0.2765246,1,1 hours
Media,C04,533,0001,Media,01,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,44.13271,310.4918,56443,1.85757732,-2.9642274,4,4 hours
Media,C04,533,0001,Media,01,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1978.14844,625.7906,56443,0.41697550,-1.0926238,4,4 hours


In [8]:
figure_path <- file.path("..","figures",data_set)
# create the output directory if it does not exist
if (!dir.exists(figure_path)) {
  dir.create(figure_path)
}

In [9]:
head(umap_data)


wells <- unique(umap_data$Metadata_Well)
# sort the wells alphabetically
wells <- sort(wells)

Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1,Metadata_timepoint,Metadata_timepoint_hours
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1426.31944,1051.7531,56443,4.50723457,0.8430272,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1278.15894,451.1779,56443,-1.14765608,-0.8359004,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,116.34923,683.6766,56443,0.05474161,-0.2473071,1,1 hours
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,399.61502,1031.3970,56443,-0.25203672,0.2765246,1,1 hours
Media,C04,533,0001,Media,01,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,44.13271,310.4918,56443,1.85757732,-2.9642274,4,4 hours
Media,C04,533,0001,Media,01,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1978.14844,625.7906,56443,0.41697550,-1.0926238,4,4 hours


In [10]:
print(dim(umap_data))
print(dim(pca_data))
print(dim(scree_data))

head(umap_data,1)
head(pca_data,1)
head(scree_data,1)

[1] 32892    37
[1] 3820888      35
[1] 100   2


Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1,Metadata_timepoint,Metadata_timepoint_hours
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
Media,C04,502,0001,Media,00,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0001,1426.319,1051.753,56443,4.507235,0.8430272,1,1 hours


Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_BF,Metadata_Image_PathName_CL488,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,PCA0,PCA1
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
Ab1-42 0.4 uM,N11,652,0002,Ab1-42,01,NA,0.4,uM,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0323_F0002,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0323_F0002,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0323_F0002,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0323_F0002,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0323_F0002,16.93757,14.57341,68519,13923115,1941383


Explained Variance,Principal Component
<dbl>,<int>
0.5872091,1


In [11]:
print(dim(umap_data))
print(dim(pca_data))

# randomly mix the order of the rows to prevent plotting bias and artifacts
umap_data <- umap_data[sample(nrow(umap_data)),]
pca_data <- pca_data[sample(nrow(pca_data)),]

print(dim(umap_data))
print(dim(pca_data))

[1] 32892    37
[1] 3820888      35
[1] 32892    37
[1] 3820888      35


In [12]:
unique(umap_data$Metadata_treatment)
umap_data$Metadata_treatment <- factor(umap_data$Metadata_treatment, levels = c(
    'Media',
    'DMSO CTL',
    'no Hoechst',

    'LPS 0.1 ug/ml',
    'LPS 1 ug/ml',
    'LPS 10 ug/ml',
    'LPS 1 ug/ml + ATP 2.5 mM',
    'LPS 1 ug/ml + Nigericin 0.1 uM',
    'LPS 1 ug/ml + Nigericin 0.5uM',
    'LPS 1 ug/ml + Nigericin 1 uM',
    'LPS 1 ug/ml + Nigericin 3uM',
    'LPS 1 ug/ml + Nigericin 5uM',

    'Flagellin 0.1 ug/ml',
    'Flagellin 1 ug/ml',
    'Flagellin 10 ug/ml',

    'Thapsigargin 0.5uM',
    'Thapsigargin 1 uM',
    'Thapsigargin 10 uM',
    'H2O2 100 nM',
    'H2O2 100 uM',
    'H2O2 500 uM',

    'Ab1-42 0.4 uM',
    'Ab1-42 2 uM',
    'Ab1-42 10 uM'
))
unique(umap_data$Metadata_treatment)


[1] "Flagellin 1 ug/ml"              "LPS 1 ug/ml + Nigericin 0.5uM" 
 [3] "H2O2 500 uM"                    "LPS 10 ug/ml"                  
 [5] "Thapsigargin 0.5uM"             "LPS 1 ug/ml + ATP 2.5 mM"      
 [7] "LPS 0.1 ug/ml"                  "DMSO CTL"                      
 [9] "Flagellin 0.1 ug/ml"            "Ab1-42 0.4 uM"                 
[11] "LPS 1 ug/ml + Nigericin 1 uM"   "H2O2 100 nM"                   
[13] "Flagellin 10 ug/ml"             "LPS 1 ug/ml + Nigericin 0.1 uM"
[15] "Thapsigargin 10 uM"             "Ab1-42 2 uM"                   
[17] "LPS 1 ug/ml + Nigericin 3uM"    "H2O2 100 uM"                   
[19] "Ab1-42 10 uM"                   "LPS 1 ug/ml + Nigericin 5uM"   
[21] "LPS 1 ug/ml"                    "Thapsigargin 1 uM"             
[23] "Media"

[1] Flagellin 1 ug/ml              LPS 1 ug/ml + Nigericin 0.5uM 
 [3] H2O2 500 uM                    LPS 10 ug/ml                  
 [5] Thapsigargin 0.5uM             LPS 1 ug/ml + ATP 2.5 mM      
 [7] LPS 0.1 ug/ml                  DMSO CTL                      
 [9] Flagellin 0.1 ug/ml            Ab1-42 0.4 uM                 
[11] LPS 1 ug/ml + Nigericin 1 uM   H2O2 100 nM                   
[13] Flagellin 10 ug/ml             LPS 1 ug/ml + Nigericin 0.1 uM
[15] Thapsigargin 10 uM             Ab1-42 2 uM                   
[17] LPS 1 ug/ml + Nigericin 3uM    H2O2 100 uM                   
[19] Ab1-42 10 uM                   LPS 1 ug/ml + Nigericin 5uM   
[21] LPS 1 ug/ml                    Thapsigargin 1 uM             
[23] Media                         
24 Levels: Media DMSO CTL no Hoechst LPS 0.1 ug/ml LPS 1 ug/ml ... Ab1-42 10 uM

In [13]:


# 9 colors
# col 1 - 3 hues
# colorgrad1 <- c("white", "grey", "#585858")
colorgrad1 <- colorRampPalette(c("lightgrey", "grey"))(3)
# col 2 - 5 hues color ramp 5 hues
colorgrad2 <- colorRampPalette(c("pink", "darkred"))(11)
# col 3 - 3 hues
colorgrad3 <- colorRampPalette(c("yellow", "brown"))(3)
# col 4 - 3 hues
colorgrad4 <- colorRampPalette(c("lightblue", "darkblue"))(6)
# col 5 - 2 hues
colorgrad5 <- colorRampPalette(c("lightgreen", "darkgreen"))(3)
# col 6 - 3 hues
colorgrad6 <- colorRampPalette(c("purple", "#2e004b"))(3)
# col 7 - 2 hues
colorgrad7 <- colorRampPalette(c("cyan", "darkcyan"))(4)
# col 8 - 2 hues
colorgrad8 <- colorRampPalette(c("#ebb676", "darkorange"))(3)
# col 9 - 3 hues
colorgrad9 <- colorRampPalette(c("magenta", "#833b83"))(3)

In [14]:
# define the colors
colors <- c(
    'Media' = colorgrad1[1],
    'DMSO CTL' = colorgrad1[2],
    'no Hoechst' = colorgrad1[3],

    'LPS 0.1 ug/ml' = colorgrad2[1],
    'LPS 1 ug/ml' = colorgrad2[2],
    'LPS 10 ug/ml' = colorgrad2[3],
    'LPS 1 ug/ml + ATP 2.5 mM' = colorgrad2[4],
    'LPS 1 ug/ml + Nigericin 0.1 uM' = colorgrad2[5],
    'LPS 1 ug/ml + Nigericin 0.5uM' = colorgrad2[6],
    'LPS 1 ug/ml + Nigericin 1 uM' = colorgrad2[7],
    'LPS 1 ug/ml + Nigericin 3uM' = colorgrad2[8],
    'LPS 1 ug/ml + Nigericin 5uM' = colorgrad2[9],

    'Flagellin 0.1 ug/ml' = colorgrad5[1],
    'Flagellin 1 ug/ml' = colorgrad5[2],
    'Flagellin 10 ug/ml' = colorgrad5[3],

    'Thapsigargin 0.5uM' = colorgrad8[1],
    'Thapsigargin 1 uM' = colorgrad8[2],
    'Thapsigargin 10 uM' = colorgrad8[3],
    
    'H2O2 100 nM' = colorgrad7[1],
    'H2O2 100 uM' = colorgrad7[2],
    'H2O2 500 uM' = colorgrad7[3],

    'Ab1-42 0.4 uM' = colorgrad4[1],
    'Ab1-42 2 uM' = colorgrad4[2],
    'Ab1-42 10 uM' = colorgrad4[3]
)

In [15]:
# order the df by time 
umap_data <- umap_data %>% arrange(Metadata_Time)
head(umap_data)

Metadata_treatment,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_CL561,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1,Metadata_timepoint,Metadata_timepoint_hours
<fct>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>
LPS 1 ug/ml + ATP 2.5 mM,G08,378,0001,LPS,00,ATP,1,ug/ml,2.5,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0152_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0152_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0152_F0001,192.7695,417.1523,39239,-0.2725063,0.8213332,1,1 hours
Flagellin 10 ug/ml,H04,419,0001,Flagellin,00,NA,10,ug/ml,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0172_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0172_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0172_F0001,1935.8476,1496.7722,49263,2.5540144,-1.2589241,1,1 hours
Flagellin 0.1 ug/ml,J06,432,0005,Flagellin,00,NA,0.1,ug/ml,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0222_F0005,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0222_F0005,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0222_F0005,190.9239,1093.7941,46820,0.6835998,0.8113541,1,1 hours
LPS 1 ug/ml + ATP 2.5 mM,G06,468,0003,LPS,00,ATP,1,ug/ml,2.5,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0150_F0003,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0150_F0003,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0150_F0003,389.6216,690.3729,35745,3.1698720,0.9466364,1,1 hours
Ab1-42 2 uM,M11,613,0001,Ab1-42,00,NA,2,uM,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0299_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0299_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0299_F0001,487.6335,418.6684,68654,0.3263702,-3.3908195,1,1 hours
Flagellin 0.1 ug/ml,J08,376,0001,Flagellin,00,NA,0.1,ug/ml,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0224_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0224_F0001,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0224_F0001,1142.8211,1759.5402,46217,-1.1130608,-0.7135559,1,1 hours


In [16]:
name_repair_function <- function(names) {
  names[1] <- paste0(names[1], "_original")
  return(names)
}
df_background <- tidyr::crossing(
    umap_data,
    Metadata_treatment = unique(umap_data$Metadata_treatment),
    .name_repair = name_repair_function
)
head(df_background)

Metadata_treatment_original,Metadata_Well,Metadata_number_of_singlecells,Metadata_FOV,Metadata_treatment1,Metadata_Time,Metadata_treatment2,Metadata_treatment1_dose,Metadata_treatment1_unit,Metadata_treatment2_dose,⋯,Metadata_Image_PathName_DNA,Metadata_Image_PathName_GSDM,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_cells_per_well,UMAP0,UMAP1,Metadata_timepoint,Metadata_timepoint_hours,Metadata_treatment
<fct>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<fct>
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,Media
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,DMSO CTL
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,no Hoechst
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,LPS 0.1 ug/ml
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,LPS 1 ug/ml
Media,C04,306,0004,Media,03,NA,NA,NA,NA,⋯,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,/gpfs/alpine1/scratch/mlippincott@xsede.org/pyroptosis_live-cell_timelapse/Wave1_data/2.illumination_correction/illum_directory/W0052_F0004,1669.406,1051.072,56443,-0.8967128,2.37334,10,10 hours,LPS 10 ug/ml


In [17]:
# get the centroids of each treatment, at each timepoint 
centroids <- umap_data %>% 
  group_by(Metadata_treatment, Metadata_timepoint) %>% 
  summarize(
    UMAP0 = mean(UMAP0),
    UMAP1 = mean(UMAP1)
  ) %>% 
  ungroup()
head(centroids)


`summarise()` has grouped output by 'Metadata_treatment'. You can override
using the `.groups` argument.


Metadata_treatment,Metadata_timepoint,UMAP0,UMAP1
<fct>,<dbl>,<dbl>,<dbl>
Media,1,0.8535414,0.1607261
Media,4,1.5595761,-0.2079760
Media,7,0.8656837,-0.1712147
Media,10,0.4046431,0.3157091
Media,13,1.1584869,-0.3718695
Media,16,0.7226481,-0.2188411


In [18]:

# plot the centroids over time
width <- 10
height <- 10
options(repr.plot.width=width, repr.plot.height=height)
centroids_umap_plot <- (
    # plot the whole dataset in grey with low alpha
    ggplot(centroids, aes(x=UMAP0, y=UMAP1))
    + geom_point(
        data = df_background,
        color = "lightgrey",
        size = 0.5,
        alpha = 0.1
    )
    + geom_point(aes(color=Metadata_treatment), size=1.5, alpha=0.9)

    + scale_color_manual(
        values = colors
    )
    + theme_bw()
    + theme(
        axis.text.x = element_text(size=14),
        axis.text.y = element_text(size=14),
        axis.title.x = element_text(size=14),
        axis.title.y = element_text(size=14),
        legend.text = element_text(size=14),
        legend.title = element_text(size=14, hjust=0.5),
        legend.position="bottom",
        legend.title.position="top"
    )
    + transition_time(Metadata_timepoint)
    + labs(title = "Time: {frame_time}")
)

n_timepoints <- length(unique(centroids$Metadata_timepoint))
fps <- 15

nframes <- n_timepoints * fps


In [19]:

anim_save(
    "../figures/pyroptosis_timelapsed_centroids.gif", 
    animate(
        centroids_umap_plot,
        fps = fps, 
        nframes = nframes,
        width = width, 
        height = height, 
        units = "in",
        res = 600
        ))

In [20]:

# plot the centroids over time
width <- 30
height <- 30
options(repr.plot.width=width, repr.plot.height=height)
centroids_umap_plot <- (
    # plot the whole dataset in grey with low alpha
    ggplot(centroids, aes(x=UMAP0, y=UMAP1))
    + geom_point(
        data = df_background,
        color = "lightgrey",
        size = 0.5,
        alpha = 0.1
    )
    + geom_point(aes(color=Metadata_treatment), size=1.5, alpha=0.9)

    + scale_color_manual(
        values = colors
    )
    + theme_bw()
    + theme(
        axis.text.x = element_text(size=14),
        axis.text.y = element_text(size=14),
        axis.title.x = element_text(size=14),
        axis.title.y = element_text(size=14),
        legend.text = element_text(size=14),
        legend.title = element_text(size=14, hjust=0.5),
        legend.position="bottom",
        legend.title.position="top"
    )
    + facet_wrap(~Metadata_treatment, ncol = 6)
    + transition_time(Metadata_timepoint)
    + labs(title = "Time: {frame_time}")
)

n_timepoints <- length(unique(centroids$Metadata_timepoint))
fps <- 15

nframes <- n_timepoints * fps


In [21]:
anim_save(
    "../figures/pyroptosis_timelapsed_centroids_facet_by_treatment.gif", 
    animate(
        centroids_umap_plot,
        fps = fps, 
        nframes = nframes, 
        width = width, 
        height = height, 
        units = "in",
        res = 600
        ))